In [2]:
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
data_clean_folder = "stage3_data_cleaning/v2"
folder_path = data_clean_folder
xlsx_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
xlsx_files

['stage3_data_cleaning/v2\\type10_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type1_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type2_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type3_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type4_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type5_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type6_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type7_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type8_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type9_label_merged_final_decoded_clean3.xlsx']

In [4]:
# df=pd.read_excel(xlsx_files[1])
# df.columns

In [5]:
# data = df
data = pd.read_excel(xlsx_files[0])
data.head()

,label,timestamp,can_id,data_length,source_node_id_decimal,service_flag,priority,message_type_decimal,destination_node_id_decimal,request_or_response,...,end_of_message,single_message_frame,transfer_ID,effective_data_0,effective_data_1,effective_data_2,effective_data_3,effective_data_4,effective_data_5,effective_data_6
0,0,0.000000,05040601,8,1,0,5,1030,-99,-99,...,0,0,20,0,0,0,0,0,-199,-199
1,0,0.000139,05040601,7,1,0,5,1030,-99,-99,...,1,0,20,0,0,0,0,0,0,-199
2,0,0.004482,05040601,8,1,0,5,1030,-99,-99,...,0,0,21,0,0,0,0,0,-199,-199
3,0,0.004641,05040601,7,1,0,5,1030,-99,-99,...,1,0,21,0,0,0,0,0,0,-199
4,0,0.009330,05040601,8,1,0,5,1030,-99,-99,...,0,0,22,0,0,0,0,0,-199,-199


In [6]:
from sklearn.model_selection import train_test_split

def feature_analysis(data):
    
    target = data['label']
    features = data.drop(['timestamp', 'label','can_id'], axis=1)
    
    X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    print(f"Train set: {X_train.shape}")
    print(f"Validation set: {X_val.shape}")
    print(f"Test set: {X_test.shape}")

    return X_train,y_train,X_val,y_val, X_test,y_test

In [7]:
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error,accuracy_score

def decision_tree(X_train, y_train,X_val,y_val,X_test,y_test):
    
    # dt_model = DecisionTreeRegressor()
    dt_model = DecisionTreeClassifier()

    dt_model.fit(X_train, y_train)
    
    y_val_pred = dt_model.predict(X_val)
    # mse_val_dt = mean_squared_error(y_val, y_val_pred)
    accuracy_val_dt = accuracy_score(y_val, y_val_pred)

    y_test_pred = dt_model.predict(X_test)
    # mse_test_dt = mean_squared_error(y_test, y_test_pred)
    accuracy_test_dt = accuracy_score(y_test, y_test_pred)

    # accuracy_val_dt = accuracy_score(y_val, y_val_pred)
    print(f"Decision Tree Validation Accuracy: {accuracy_val_dt}")
    print(f"Decision Tree test Accuracy: {accuracy_test_dt}")

    # print(f"Decision Tree Validation MSE: {mse_val_dt}")
    # print(f"Decision Tree Test MSE: {mse_test_dt}")

    # return mse_val_dt,mse_test_dt,accuracy_val_dt,accuracy_test_dt
    return accuracy_val_dt,accuracy_test_dt

def rf_tree(X_train, y_train,X_val,y_val,X_test,y_test):
    # rf_model = RandomForestRegressor()
    rf_model = RandomForestClassifier()
    
    rf_model.fit(X_train, y_train)
    
    y_val_pred = rf_model.predict(X_val)
    # mse_val_rf = mean_squared_error(y_val, y_val_pred)
    accuracy_val_rf = accuracy_score(y_val, y_val_pred)

    y_test_pred = rf_model.predict(X_test)
    # mse_test_rf = mean_squared_error(y_test, y_test_pred)
    accuracy_test_rf = accuracy_score(y_test, y_test_pred)
    print(f"Decision Tree Validation Accuracy: {accuracy_val_rf}")
    print(f"Decision Tree test Accuracy: {accuracy_test_rf}")

    # print(f"Decision Tree Validation MSE: {mse_val_rf}")
    # print(f"Decision Tree Test MSE: {mse_test_rf}")

    # return mse_val_rf,mse_test_rf,accuracy_val_rf,accuracy_test_rf
    return accuracy_val_rf,accuracy_test_rf


In [8]:
results_df = pd.DataFrame()
results_list = []
for file_path in xlsx_files:
    # print(file_path)
    results = {}
    label_key = os.path.basename(file_path).split('.')[0][:-28]
    print(label_key)
    results['dataset'] = label_key
    
    data = pd.read_excel(file_path)
    X_train,y_train,X_val,y_val, X_test,y_test = feature_analysis(data)

    # mse_val_dt,mse_test_dt,accuracy_val_dt,accuracy_test_dt = decision_tree(X_train, y_train,X_val,y_val,X_test,y_test)
    # mse_val_rf,mse_test_rf,accuracy_val_rf,accuracy_test_rf = rf_tree(X_train, y_train,X_val,y_val,X_test,y_test)


    accuracy_val_dt,accuracy_test_dt = decision_tree(X_train, y_train,X_val,y_val,X_test,y_test)
    accuracy_val_rf,accuracy_test_rf = rf_tree(X_train, y_train,X_val,y_val,X_test,y_test)

    # results['DT_MSE_VAL'] = mse_val_dt
    results['DT_ACC_VAL'] = accuracy_val_dt
    # results['DT_MSE_TEST'] = mse_test_dt
    results['DT_ACC_Test'] = accuracy_test_dt

    # results['RF_MSE_VAL'] = mse_val_rf
    results['RF_ACC_VAL'] = accuracy_val_rf
    # results['RF_MSE_TEST'] = mse_test_rf
    results['RF_ACC_Test'] = accuracy_test_rf

    
    # results_df = results_df.append(results, ignore_index=True)
    print(results)
    results_list.append(results)
    

results_df = pd.DataFrame(results_list)


type10_label
Train set: (145166, 19)
Validation set: (31107, 19)
Test set: (31107, 19)
Decision Tree Validation Accuracy: 0.9079306908412897
Decision Tree test Accuracy: 0.9111132542514546
Decision Tree Validation Accuracy: 0.9052624811135758
Decision Tree test Accuracy: 0.9096023403092551
{'dataset': 'type10_label', 'DT_ACC_VAL': 0.9079306908412897, 'DT_ACC_Test': 0.9111132542514546, 'RF_ACC_VAL': 0.9052624811135758, 'RF_ACC_Test': 0.9096023403092551}
type1_label
Train set: (145500, 19)
Validation set: (31179, 19)
Test set: (31179, 19)
Decision Tree Validation Accuracy: 0.987363289393502
Decision Tree test Accuracy: 0.9875236537413002
Decision Tree Validation Accuracy: 0.987363289393502
Decision Tree test Accuracy: 0.9875236537413002
{'dataset': 'type1_label', 'DT_ACC_VAL': 0.987363289393502, 'DT_ACC_Test': 0.9875236537413002, 'RF_ACC_VAL': 0.987363289393502, 'RF_ACC_Test': 0.9875236537413002}
type2_label
Train set: (93919, 19)
Validation set: (20125, 19)
Test set: (20126, 19)
Decisio

In [9]:
results_df

,dataset,DT_ACC_VAL,DT_ACC_Test,RF_ACC_VAL,RF_ACC_Test
0,type10_label,0.907931,0.911113,0.905262,0.909602
1,type1_label,0.987363,0.987524,0.987363,0.987524
2,type2_label,0.985242,0.983653,0.985242,0.983653
3,type3_label,0.996725,0.995881,0.996928,0.996253
4,type4_label,0.995851,0.994952,0.994902,0.995202
5,type5_label,0.978295,0.977964,0.977816,0.977964
6,type6_label,0.998342,0.997707,0.997983,0.997928
7,type7_label,0.916553,0.918178,0.916781,0.918377
8,type8_label,0.997993,0.998194,0.998144,0.998445
9,type9_label,0.912377,0.917557,0.912058,0.917383


In [10]:
output_file = 'results_DT_RF_v2.xlsx'
results_df.to_excel(output_file, index=False)

In [ ]:


# Initialize the model
dt_model = DecisionTreeRegressor()

# Train the model
dt_model.fit(X_train, y_train)

# Predict on validation set
y_val_pred = dt_model.predict(X_val)

# Evaluate the model
mse_val_dt = mean_squared_error(y_val, y_val_pred)
print(f"Decision Tree Validation MSE: {mse_val_dt}")


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
rf_model = RandomForestRegressor()

# Train the model
rf_model.fit(X_train, y_train)

# Predict on validation set
y_val_pred = rf_model.predict(X_val)

# Evaluate the model
mse_val_rf = mean_squared_error(y_val, y_val_pred)
print(f"Random Forest Validation MSE: {mse_val_rf}")


In [ ]:
from sklearn.svm import SVR

# Initialize the model
svr_model = SVR()

# Train the model
svr_model.fit(X_train, y_train)

# Predict on validation set
y_val_pred = svr_model.predict(X_val)

# Evaluate the model
mse_val_svr = mean_squared_error(y_val, y_val_pred)
print(f"Support Vector Regressor Validation MSE: {mse_val_svr}")
